In [16]:
import uuid
import random
import pandas as pd

In [17]:
user_df = pd.read_csv("adjusted_datasets/users.csv", encoding="utf-8", encoding_errors="replace") 
flight_df = pd.read_csv("converted_datasets/flights_new.csv", encoding="utf-8", encoding_errors="replace") 
restaurant_df = pd.read_csv("converted_datasets/restaurants_new.csv", encoding="utf-8", encoding_errors="replace") 
city_df = pd.read_csv("converted_datasets/cities_new.csv", encoding="utf-8", encoding_errors="replace") 
hotel_df = pd.read_csv("converted_datasets/hotels_new.csv", encoding="utf-8", encoding_errors="replace") 

In [18]:
import uuid
import random
import pandas as pd

history_data = []

for user_id in user_df["user_id"]:
    history_id = "HIST" + uuid.uuid4().hex[:8]  # Generate unique history ID

    # Pick a random city and its corresponding country
    if not city_df.empty:
        selected_city_row = city_df.sample(n=1).iloc[0]
        city = selected_city_row.get("City", None)
        country = selected_city_row["Country"]
    else:
        city = None
        country = random.choice(city_df["Country"].unique())  # Pick a random country

    # Pick flights where the departure/arrival city or country matches
    if city:
        flight_dep_options = flight_df[flight_df["City Dep"] == city]["flight_id"]
        flight_arr_options = flight_df[flight_df["City Arr"] == city]["flight_id"]
    else:
        flight_dep_options = flight_df[flight_df["Country Dep"] == country]["flight_id"]
        flight_arr_options = flight_df[flight_df["Country Arr"] == country]["flight_id"]

    flight_dep = flight_dep_options.sample(n=1).values[0] if not flight_dep_options.empty else None
    flight_arr = flight_arr_options.sample(n=1).values[0] if not flight_arr_options.empty else None

    # Randomize length of stay (3-14 days)
    length_of_stay = random.randint(3, 14)

    # Select restaurants from the chosen city
    available_restaurants = restaurant_df[restaurant_df["City"] == city]["restaurant_id"].tolist() if city else []
    num_restaurants = length_of_stay * random.choice([2, 2, 2, 3])  # Mostly 2, sometimes 3 per day
    restaurants = random.sample(available_restaurants, min(num_restaurants, len(available_restaurants))) if available_restaurants else []

    # Select hotels (preferably 1, but change every 3-4 days if length_of_stay > 4)
    available_hotels = hotel_df[hotel_df["City"] == city]["hotel_id"].tolist() if city else []
    if length_of_stay <= 4:
        hotels = [random.choice(available_hotels)] if available_hotels else []
    else:
        num_hotels = max(1, length_of_stay // random.randint(3, 4))  # Change hotels every 3-4 days
        hotels = random.sample(available_hotels, min(num_hotels, len(available_hotels))) if available_hotels else []

    # Append the history entry
    history_data.append({
        "history_id": history_id,
        "user_id": user_id,
        "city": city if city else "N/A",
        "country": country,
        "flight_dep": flight_dep,
        "flight_arr": flight_arr,
        "length_of_stay": length_of_stay,
        "restaurants": restaurants,
        "hotels": hotels
    })

# Convert to DataFrame
df = pd.DataFrame(history_data)

print(df.to_markdown())

|    | history_id   | user_id                              | city           | country              | flight_dep   | flight_arr   |   length_of_stay | restaurants                                                                                                                                                                                                                                                                                                  | hotels                                  |
|---:|:-------------|:-------------------------------------|:---------------|:---------------------|:-------------|:-------------|-----------------:|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------|
|  0 | HIS

In [19]:
import os

# Define the folder path
folder_path = "adjusted_datasets"

# Ensure the folder exists
os.makedirs(folder_path, exist_ok=True)

# Define the full file path
file_path = os.path.join(folder_path, "histories.csv")

# Save the DataFrame as a CSV file
df.to_csv(file_path, index=False, encoding='utf-8')

print(f"CSV file saved successfully at: {file_path}")

CSV file saved successfully at: adjusted_datasets\histories.csv
